In [2]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import panel as pn
pn.extension()

sys.path.append(os.path.abspath('../'))
os.environ["NUMBA_CUDA_DRIVER"] = "C:\\Windows\\System32\\nvcuda.dll"
from util.data_utils import load_csv_folder
from util.viz_utils import line_chart

In [3]:
# 加载数据
data_dir = '../../data/sat1'  # 路径可根据实际情况调整
dataframes = load_csv_folder(data_dir)
file = list(dataframes.keys())[0]  # 选择第一个文件演示
df = dataframes[file]
print(f'加载数据文件: {file}, 行数: {len(df)}')
df.head()

加载数据文件: 147_2025031706171299310.csv, 行数: 2128810


,147_2025031706171299310.csv
timestamp,
2024-01-01 00:07:36.286,20
2024-01-01 00:07:50.786,20
2024-01-01 00:08:05.786,20
2024-01-01 00:08:20.285,20
2024-01-01 00:08:35.285,20


In [4]:
chart=line_chart(dataframes)
chart

d:\CondaEnvs\envs\pytorch_env\Lib\site-packages\holoviews\operation\downsample.py:67: RuntimeWarning: overflow encountered in scalar multiply
  + (prev_x * avg_next_y - avg_next_x * prev_y)
d:\CondaEnvs\envs\pytorch_env\Lib\site-packages\holoviews\operation\downsample.py:67: RuntimeWarning: overflow encountered in scalar multiply
  + (prev_x * avg_next_y - avg_next_x * prev_y)


BokehModel(combine_events=True, render_bundle={'docs_json': {'ca024d3d-9d87-483d-a745-ca57c7c1ec21': {'version…

d:\CondaEnvs\envs\pytorch_env\Lib\site-packages\holoviews\operation\downsample.py:67: RuntimeWarning: overflow encountered in scalar multiply
  + (prev_x * avg_next_y - avg_next_x * prev_y)
d:\CondaEnvs\envs\pytorch_env\Lib\site-packages\holoviews\operation\downsample.py:67: RuntimeWarning: overflow encountered in scalar multiply
  + (prev_x * avg_next_y - avg_next_x * prev_y)
d:\CondaEnvs\envs\pytorch_env\Lib\site-packages\holoviews\operation\downsample.py:67: RuntimeWarning: overflow encountered in scalar multiply
  + (prev_x * avg_next_y - avg_next_x * prev_y)
d:\CondaEnvs\envs\pytorch_env\Lib\site-packages\holoviews\operation\downsample.py:67: RuntimeWarning: overflow encountered in scalar multiply
  + (prev_x * avg_next_y - avg_next_x * prev_y)
d:\CondaEnvs\envs\pytorch_env\Lib\site-packages\holoviews\operation\downsample.py:67: RuntimeWarning: overflow encountered in scalar multiply
  + (prev_x * avg_next_y - avg_next_x * prev_y)
d:\CondaEnvs\envs\pytorch_env\Lib\site-packages\ho

In [5]:
# 切片函数
def slice_data(df, start, end):
    return df[start:end]
# 滑窗函数
def sliding_window(df, window_size, step_size):
    for i in range(0, len(df) - window_size + 1, step_size):
        yield df[i:i+window_size]

In [6]:
import stumpy
import numpy as np
from numba import cuda

# 获取上半年的数据
half = {k: v for k, v in dataframes.items() if v.index.month < 7}

# 计算单维mp
devices = [device.id for device in cuda.list_devices()]
mps = {}
for file in half.keys():
    data=dataframes[file].values.astype(np.float64).T
    print(data)
    mps[file]=stumpy.gpu_stump(data[0], m=20)

[[20. 20. 20. ... 20. 20. 20.]]
[[20. 20. 20. ... 20. 20. 20.]]
[[51.13 51.13 51.13 ... 50.56 50.57 50.56]]


KeyboardInterrupt: 

In [ ]:
m = 500  # 窗口长度，可根据实际数据调整
ts = df['value'].values
mp = stumpy.stump(ts, m=m)
L = m
excl_factor = 1
stream = stumpy.floss(mp, ts, m=m, L=L, excl_factor=excl_factor)
cac_1d = stream.cac_1d_
print(f'arc curve长度: {len(cac_1d)}')

In [ ]:
fig, axs = plt.subplots(2, sharex=True, figsize=(16, 6), gridspec_kw={'hspace': 0.1})
axs[0].plot(df.index[m-1:], ts[m-1:], label='原始数据')
axs[0].set_ylabel('数值')
axs[0].legend()
axs[1].plot(df.index[m-1:], cac_1d, color='orange', label='FLOSS arc curve')
axs[1].set_ylabel('arc curve')
axs[1].set_xlabel('时间')
axs[1].legend()
plt.suptitle(f'FLOSS分段arc curve ({file})')
plt.show()

In [ ]:
from scipy.signal import find_peaks
peaks, _ = find_peaks(-cac_1d, distance=m)  # 距离至少m，避免过密
print(f'分段点索引: {peaks}')

# 可视化分段点
fig, axs = plt.subplots(2, sharex=True, figsize=(16, 6), gridspec_kw={'hspace': 0.1})
axs[0].plot(df.index[m-1:], ts[m-1:], label='原始数据')
axs[0].vlines(df.index[m-1:][peaks], ymin=ts.min(), ymax=ts.max(), color='red', linestyle='dashed', label='分段点')
axs[0].set_ylabel('数值')
axs[0].legend()
axs[1].plot(df.index[m-1:], cac_1d, color='orange', label='FLOSS arc curve')
axs[1].vlines(df.index[m-1:][peaks], ymin=cac_1d.min(), ymax=cac_1d.max(), color='red', linestyle='dashed', label='分段点')
axs[1].set_ylabel('arc curve')
axs[1].set_xlabel('时间')
axs[1].legend()
plt.suptitle(f'FLOSS分段及分段点 ({file})')
plt.show()